# Zernikes

Prysm supports two flavors of Zernike polynomials; the Fringe set up to the 49th term, and the Zemax Standard set up to the 48th term. They have identical interfaces, so only one will be shown here.

Zernike notations are a subclass of [Pupil](./Pupils.ipynb), so they support the same arguments to `__init__`,

In [ ]:
%load_ext autoreload
%autoreload 2
from prysm import FringeZernike, StandardZernike
p = FringeZernike(samples=123, dia=456.7, wavelength=1.0, opd_unit='nm', mask='dodecagon')

There are additional keyword arguments for each term, and the base (0 or 1) can be supplied. With base 0, the terms start at Z0 and range to Z48. With base 1, they start at Z1 and range to Z49 (or Z48, for Standard Zernikes). The Fringe set can also be used with unit variance via the `norm` keyword argument. Both notations also have nice print statements,

In [ ]:
p2 = FringeZernike(Z4=1, Z9=1, Z48=1, base=0, norm=True)
p2

Notice that the RMS value is equal to sqrt(1^2 + 1^2 + 1^2) = sqrt(3) = 1.732 ~= 1.722. The difference of ~1% is due to the array sizes used by prysm by default, if increased, e.g. by adding `samples=1204` to the constructor, the value would converge to the analytical one.

A Zernike pupil can also be initalized with an iterable (list, tuple...) of coefficients,

In [ ]:
import numpy as np
terms = np.random.rand(49)  # 49 zernike coefficients, e.g. from a wavefront sensor
fz3 = FringeZernike(terms)

FringeZernike has many features StandardZernike does not. At the module level are two functions,

In [ ]:
from prysm.fringezernike import fzname, fzset_to_magnitude_angle

`fzname` takes an index and optional base (default 0) kwarg and returns the name of that term. `fzset_to_magnitude_angle` takes a non-sparse iterable of fringe zernike coefficients, starting with piston, and returns a dictionary with keywords of terms (e.g. “Primary Astigmatism”) and items that are length 2 tuples of (magnitude, angle) where magnitude is in the same units as the zernike coefficients and the angle is in degrees. `fzset_to_magnitude_angle`’s output format can be seen below on the example of `FringeZernike.magnitudes`.

FringeZernike instances have a `truncate` method which discards terms with indices higher than n. For example,

In [ ]:
fz3.truncate(16)

this is less efficient, however, than simply slicing the coefficient vector,

In [ ]:
fz4 = FringeZernike(terms[:16])

and this slicing alternative should be used when one is sensitive to performance.

The top few terms may be extracted,

In [ ]:
fz4.top_n(5)

or the terms listed by their pairwise magnitudes and clocking angles,

In [ ]:
fz4.magnitudes

These things may be (bar) plotted;

In [ ]:
fz4.barplot(orientation='h', buffer=1, zorder=3)
fz4.barplot_magnitudes(orientation='h', buffer=1, zorder=3)
fz4.barplot_topn(n=5, orientation='h', buffer=1, zorder=3)

`orientation` controls the axis on which the terms are enumerated. `h` results in vertical bars, `v` is also accepted, as are horizontal and vertical. `buffer` is the number of terms’ worth of spaces left on each side. The default of 1 leaves a one bar margin. `zorder` is passed to matplotlib – the default of 3 places the bars above any gridlines, which is an aesthetic choice. Matplotlib has a general default of 1.

If you would like direct access to the underlying functions, there are two paths. `prysm._zernike` contains functions for the first 49 (Fringe ordered) Zernike polynomials, for example

In [ ]:
from prysm._zernike import defocus

each of these takes arguments of (rho, phi). They have names which end with _x, or _00 and _45 for the terms which have that naming convention.

Perhaps more convenient is a dictionary which numbers them,

In [ ]:
from prysm._zernike import _zernikes
from prysm.fringezernike import zernmap
zfunction = _zernikes[zernmap[8]]
zfunction, zfunction.name, zfunction.norm

If these will always be evaluted over a square region enclosing the unit circle, a cache is available to speed computation,

In [ ]:
from prysm.fringezernike import zcache
zcache(8, norm=True, samples=128)  # base 0
zcache.clear()  # you should never have to do this unless you want to release memory

This cache instance is used internally by prysm, if you modify the returned arrays in-place, you probably won’t like the result. You can create your own independent instance,

In [ ]:
from prysm.fringezernike import FZCache
my_fzcache = FZCache()

See [Pupils](./Pupils.ipynb) for information about general pupil functions. Below, the Fringe type of Zernike description has its full documentation printed.